In [28]:
using CUDA
using ProgressMeter
using DelimitedFiles
include("../src/forces_func.jl")
include("../src/struct_data.jl")
include("../src/test.jl")
include("../src/forces.jl")
include("../src/neighbor.jl")
include("../src/run_event.jl")
include("../src/sphere.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

In [29]:
parameters = Model(
    Cubic(
        μ₁       = 0.003,
        rₘᵢₙ     = 2,
        rₘₐₓ     = 3
    ),
    Contractile(
        fₚ       = 0.001
    ),
    Time(
        t_f      = 50000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 50,
        nn       = 12
    ),
    Geometry(
        R_agg    = 9,
        position = [
            0.0    0.0    0.0;
            0.0    0.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "After-Eq.xyz"
    )
)

## Var before func
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(parameters.Force::Cubic, r)


 Loading Force Function
  0.000013 seconds (17 CPU allocations: 1.237 KiB)


force (generic function with 1 method)

In [30]:
function generate_random_packaging(R_agg, m::Model)
    R_sup = R_agg
    while true
        println("--------------------------------------------------")
        println("R_Agg = $(R_sup)")
        println("Random Packaging")
        X = sphere(R_sup)

        # Generating Struct
        global Agg = Aggregate(Neighbor(50, 12), PositionCell(X))

        # Simulation One Aggregate
        one_aggregate(false, m, Cubic(1.0, 2.0, 3.0), Agg)

        Size = Float32(
                (findmax(Agg.Position.X[:,1])[1] - findmin(Agg.Position.X[:,1])[1])/2 + 1
            ) 

        println("---------")
        println("R_Agg_Calc = $(Size)")

        if Size < R_agg
            R_sup += + 0.5
            println("---------")
            println("Adding R_sup")
            println("*********")
        else
            break
        end
    end

    open("$(R_agg).xyz", "w") do f
        write(f, "$(size(Agg.Position.X, 1))\n")
        write(f, "Random Packaging ($(R_agg)).xyz\n")
        writedlm(f,hcat(repeat([1.0],size(Agg.Position.X, 1)), Matrix(Agg.Position.X)), ' ')
    end
    
end

generate_random_packaging (generic function with 1 method)

In [31]:
R_agg = 9
generate_random_packaging(R_agg, parameters)

--------------------------------------------------
R_Agg = 9
Random Packaging


Progress:  83%|████████████████████▉    |  ETA: 0:00:02

Not Enought
Finding Equilibrium in one Aggregate


Progress: 100%|█████████████████████████| Time: 0:01:20


---------
R_Agg_Calc = 7.6417694
---------
Adding R_sup
*********
--------------------------------------------------
R_Agg = 9.5
Random Packaging


...

---------
R_Agg_Calc = 8.7022085
---------
Adding R_sup
*********
--------------------------------------------------
R_Agg = 10.5
Random Packaging


Progress:  82%|████████████████████▍    |  ETA: 0:00:07

Not Enought
Finding Equilibrium in one Aggregate


Progress: 100%|█████████████████████████| Time: 0:01:28


---------
R_Agg_Calc = 9.099396


In [32]:
Y = Float32.(readdlm("$(R_agg).xyz")[3:end,2:end]) |> cu

473×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  1.37769    0.848875   0.583492
  3.3848     2.15684   -0.848144
 -1.34812    0.630537   4.85958
 -0.36257   -1.646     -4.35787
 -4.96672    0.706809  -2.11685
  0.657794  -0.222841   7.52348
 -0.255475  -0.168784  -0.0928241
  2.23549   -1.05008    2.9033
 -2.56616   -0.644906  -0.0335353
  0.92864   -0.590991   1.48165
  1.30733    0.996464  -1.20406
 -3.5386    -1.68364    3.39707
  2.60164   -0.606794  -7.32309
  ⋮                    
  6.0948    -4.20577    3.14182
  2.32265   -3.82247   -4.96494
 -1.3934     5.06349   -3.5387
  2.12733    4.72717    5.31348
  6.88271    2.45228   -0.670402
 -1.5244    -7.01683    2.76892
 -4.16377   -5.62851   -0.187829
 -0.14622   -4.81759   -5.53579
  3.96768    1.70834    4.20709
 -1.79841   -3.31294   -1.62391
  5.43151   -1.61526   -6.20102
 -1.2949    -5.78945    1.57864

In [33]:
run(`paraview 9.xyz`)

Process(`paraview 9.xyz`, ProcessExited(0))